# Continuous Birth-Death Mean-Field Example

This notebook demonstrates a continuous birth-death process with mean-field interactions.

In this model:
- Birth rate depends on population: λ(N) = λ₀ * N
- Death rate depends on population: μ(N) = μ₀ * N + α * N²
- The α * N² term represents competition/mean-field interaction

This shows how the same Gillespie algorithm can handle more complex rate functions.

In [ ]:
using Random
using MonteCarloX
using StatsBase

## Define the Mean-Field System

In [ ]:
# Birth-Death process with mean-field interaction
mutable struct MeanFieldBirthDeath <: AbstractSystem
    N::Int              # Population size
    λ₀::Float64         # Base birth rate
    μ₀::Float64         # Base death rate
    α::Float64          # Competition strength
    rates::Vector{Float64}  # [birth_rate, death_rate]
    
    function MeanFieldBirthDeath(N0::Int, λ₀::Float64, μ₀::Float64, α::Float64)
        birth_rate = λ₀ * N0
        death_rate = μ₀ * N0 + α * N0^2
        rates = [birth_rate, death_rate]
        new(N0, λ₀, μ₀, α, rates)
    end
end

In [ ]:
# Update function with mean-field rates
function update_rates!(sys::MeanFieldBirthDeath)
    sys.rates[1] = sys.λ₀ * sys.N                    # Birth: λ₀ * N
    sys.rates[2] = sys.μ₀ * sys.N + sys.α * sys.N^2  # Death: μ₀ * N + α * N²
end

function modify!(sys::MeanFieldBirthDeath, event::Int)
    if event == 1  # Birth
        sys.N += 1
    elseif event == 2  # Death
        sys.N -= 1
    end
    update_rates!(sys)
    return nothing
end

## Setup and Run Simulation

In [ ]:
# Create system with mean-field interaction
rng = MersenneTwister(123)
N0 = 50
λ₀ = 0.5
μ₀ = 0.3
α = 0.01  # Competition strength

sys = MeanFieldBirthDeath(N0, λ₀, μ₀, α)

# Create Gillespie algorithm
sim = init(rng, Gillespie(), sys.rates)

println("Initial population: $(sys.N)")
println("Base birth rate λ₀: $(sys.λ₀)")
println("Base death rate μ₀: $(sys.μ₀)")
println("Competition strength α: $(sys.α)")
println("\nExpected equilibrium: N* ≈ $(round((λ₀ - μ₀) / α, digits=1))")

In [ ]:
# Setup measurements
T = 200.0
measurements = Measurements([
    :population => (s -> s.N) => Float64[],
    :birth_rate => (s -> s.rates[1]) => Float64[],
    :death_rate => (s -> s.rates[2]) => Float64[]
], collect(0:1.0:T))

# Initial measurement
t = 0.0
measure!(measurements, sys, t)

In [ ]:
# Run simulation
while t <= T && sys.N > 0
    dt, event = next(sim)
    t += dt
    
    # Measure before modifying
    measure!(measurements, sys, t)
    
    # Update system (this also updates rates)
    modify!(sys, event)
    
    # Update rates in simulation
    sim.event_handler.list_rate[:] = sys.rates
end

println("\nSimulation complete")
println("Final time: $(round(t, digits=2))")
println("Final population: $(sys.N)")

## Analyze Results

In [ ]:
pop_data = measurements[:population].data
birth_data = measurements[:birth_rate].data
death_data = measurements[:death_rate].data

# Analyze equilibrium (last 20% of data)
eq_start = Int(floor(0.8 * length(pop_data)))
eq_pop = pop_data[eq_start:end]

println("\nPopulation statistics:")
println("  Overall mean: $(round(mean(pop_data), digits=2))")
println("  Equilibrium mean: $(round(mean(eq_pop), digits=2))")
println("  Equilibrium std: $(round(std(eq_pop), digits=2))")

# Compare with theoretical equilibrium
N_theory = (λ₀ - μ₀) / α
println("\nTheoretical equilibrium: $(round(N_theory, digits=2))")
println("Difference: $(round(abs(mean(eq_pop) - N_theory), digits=2))")

# Check rate balance at equilibrium
eq_birth = mean(birth_data[eq_start:end])
eq_death = mean(death_data[eq_start:end])
println("\nEquilibrium rates:")
println("  Birth rate: $(round(eq_birth, digits=3))")
println("  Death rate: $(round(eq_death, digits=3))")
println("  Balance: $(round(abs(eq_birth - eq_death), digits=3))")

## Key Insights

This example demonstrates:
1. **Mean-field interactions**: The α * N² term creates density-dependent death rate
2. **Equilibrium**: The system reaches a stable population where birth and death rates balance
3. **Fluctuations**: Stochastic dynamics cause fluctuations around equilibrium
4. **API consistency**: Same Gillespie/KMC framework works for complex rate functions